### Importy:

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

from dash.dependencies import Input, Output,State
from jupyter_dash import JupyterDash

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import scipy.stats as st
from scipy.stats._continuous_distns import _distn_names

from statsmodels.distributions.empirical_distribution import ECDF
from datetime import date, datetime

import quandl
quandl.ApiConfig.api_key = "YOUR API KEY"

<ipython-input-1-049e189358b8>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-049e189358b8>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


ModuleNotFoundError: No module named 'dash_bootstrap_components'

### Ważne!

Program wykorzystuje **listę dostępnych tickerów** (należy ściągnąć excela z listą z [dokumentacji WIKI/ na Quandl](https://www.quandl.com/databases/WIKIP/documentation?anchor=companies) i wrzucić do folderu z programem pod nazwą *WIKI_PRICES.csv*): 

In [ ]:
all_wiki_tickers= pd.read_csv("WIKI_PRICES.csv") # wszystkie akcje do wyboru z excela
all_dists = _distn_names # wszystkie rozkłady do wyboru ze scipy.stats
all_wiki_tickers.sample(5)

### Program:

Po puszczeniu kolejnej komórki powinien pojawić się pod nią link do apki.

In [ ]:
# Inicjalizacja apki
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])
plotly_template = 'none'
empty_plot = px.line( x=[0], y=[0],template=plotly_template)

# Grupa kontrolek z ustawieniami
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dcc.Markdown("Rodziny do sprawdzenia (`scipy.stats`):"),
                dcc.Dropdown(id='dist_picker',
                             options=[{'label': x, 'value': x} for x in all_dists],
                             value=['norm','t','cauchy'],
                             multi=True)
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Okres:"),
                dbc.Row([dcc.DatePickerRange(id='date_picker',
                                             max_date_allowed=datetime.today().date(),
                                             start_date=date(2016, 12, 1),end_date=date(2017, 1, 1),
                                             display_format='DD MMM, YYYY'),
                         dbc.Button([dbc.Spinner(html.Div(id="go_spinner"), size='sm')], id="go_button", color="primary"),
                         dbc.Alert("Dane niedostępne przy zadanych parametrach.", color='danger', id="data_alert",dismissable=True,is_open=False)
                        ])
                
            ]
        ),
    ],
    body=True,
)

#Opis programu
Overview = dbc.Card(
    [
        dbc.FormGroup(
        [
            html.H6("Model Blacka-Scholesa zakłada normalność log-zwrotów akcji. Czy w rzeczywistości istotnie tak jest?"),
            
            dcc.Markdown('''
            * Program zaciąga dane z serwisu Quandl, dopasowuje wybrane rodziny rozkładów i ilustruje najlepsze dopasowanie.
            * Rozkłady teoretyczne dopasowywane są metodą największej wiarogodności. 'Najlepsze' dopasowanie wybierane jest za pomocą SSE między dystrybuanta dopasowaną a empiryczną.
            * Dla każdej pary (akcja-rozkład) przeprowadzany jest test KS zgodności rozkładu empirycznego z dopasowanym teoretycznym. Wykres słupkowy po prawej stronie ilustruje liczbę odrzuceń KS testu (na poziomie istotności 5%) w danej rodzinie.
            
            '''),
            html.H6("Obserwacje:"),
            dcc.Markdown('''
            * Na długich okresach czasu prawie zawsze odrzucana jest hipoteza normalności log-zwrotów.
            * Na krótkich okresach (~miesiąc) rozkłady: normalny, cauchy'ego często dopasowują się lepiej niż rozkład t-studenta.
            * Na długich okresach rozkłady t-studenta i uogólniony normalny wydają się najlepiej dopasowywać do danych.
            '''),
            html.P("Typ rozkładu odpowiedni do modelowania logzwrotów wydaje się więc być niejednoznaczny i zależny od rozpatrywanego horyzontu czasowego.")
            
        ])
    ],
    body=True,
)


# Layout całej apki
app.layout = dbc.Container(
    [
        html.Br(),
        html.H1("Model Blacka-Scholesa"),
        html.Hr(),

        dbc.Row([
            #Col1
            dbc.Col([
                html.Br(),
                html.Br(),
                html.Br(),
                
                html.H3("Ustawienia programu:"),
                controls,
                               
                html.Br(),
                html.Br(),
                html.Br(),
                html.Br(),
                
                html.H3("Model a rzeczywistość."),
                Overview],
            md=4),

            #Col2
            dbc.Col([
                dbc.Row([
                    
                    dbc.Col([
                        html.H4("Najlepsze dopasowanie:"),
                        html.H6("Która rodzina daje najniższe SSE?"),
                        dcc.Dropdown(id='fit_check_picker',
                                     options=[{'label': x, 'value': x} for x in all_wiki_tickers['ticker']],
                                     value=['MSFT'],
                                     multi=False),
                        dcc.Graph(id="best_fit_chart")],md=5),
                    
                    dbc.Col([
                        html.H4("Rodziny rozkładów:"),
                        html.H6("Ile razy rodzina była najlepsza? Ile razy była odrzucona?"),
                        dcc.Dropdown(id='ticker_picker',
                                     options=[{'label': x, 'value': x} for x in all_wiki_tickers['ticker']],
                                     value=['MSFT','AMZN','GS'],
                                     multi=True),
                        dcc.Graph(id="histogram_chart")],md=7),
                    ]),
                
                html.H4("Log-zwroty wybranych akcji (akumulowane)"),
                dcc.Graph(id="price_chart")], md=7)
            ])

        ],
    fluid=True,
)

# Callback do zaciągania danych i plotowania wykresów
@app.callback(
    [Output("data_alert", "is_open"),
     Output("price_chart", "figure"),
     Output("histogram_chart", "figure"),
     Output("fit_check_picker", "value"),
     Output("go_spinner", "children")],
    Input("go_button", "n_clicks"),
    [State('ticker_picker','value'),
     State('dist_picker','value'),
     State('date_picker', 'start_date'),
     State('date_picker', 'end_date')])  
def main_fcn(n_clicks, ticker_list, dist_list, start_date, end_date):
    
    # Zaciąganie danych
    df = get_logrets(ticker_list, start_date, end_date)
    
    # data quality check
    try: assert not df.empty
    except: return(True,
                   empty_plot,
                   empty_plot,
                   'MSFT',"Update!")
    # Wykres logzwrotów
    stock_plot = plot_stock_logrets(df, ticker_list)
    
    # Dopasowania i testy
    best_fits, ks_pvalues, sse = perform_checks(df, dist_list)

    # Wykres slupkowy KS-testów
    ks_plot = visualize_fitted_families(best_fits, ks_pvalues, ticker_list,  dist_list)
    
    return  False, stock_plot, ks_plot, df.columns[0], "Update!"


# Osobny callback dla wykresu Best Fit, żeby nie zaciągać od nowa miliona danych, tylko 1 akcję
@app.callback(
    Output("best_fit_chart","figure"),
    Input("fit_check_picker","value"),
    [State('dist_picker','value'),
     State('date_picker', 'start_date'),
     State('date_picker', 'end_date')]
)
def display_best_fit(ticker, dist_list, start_date, end_date):
        
    try: assert len(ticker)>0
    except: return empty_plot
    
    # Dane
    df = get_logrets(ticker, start_date, end_date)
    try: assert not df.empty
    except: return empty_plot

    # Testy i fitting
    best_fits, ks_pvalues, sse = perform_checks(df, dist_list)
    
    # Wykres best fit
    best_fit_plot = plot_best_dist(ticker, best_fits, df)
    return best_fit_plot


def get_logrets(ticker_list, start_date, end_date):
    
    # Żeby nie rozbilo tickera na litery, jak się poda jednego zamiast listy
    if isinstance(ticker_list, str):
        ticker_list = [ticker_list]
        
    # .11 = adjusted close
    df = quandl.get(["WIKI/"+x+".11" for x in ticker_list],  start_date = str(start_date), end_date = str(end_date))
    
    # kolumny nazwij jak ticker
    df.columns = [(c.split(" ")[0]).split("/")[1] for c in df.columns]
    
    # czy dla jakiejs akcji nie ma wql danych?
    missing_cols_check = (df.isna().mean()==1).any()
    if missing_cols_check: return df.dropna() # jak tak, to zwracamy pustą tabele, zeby raisowac error
    
    #logzwroty
    df = np.log(df).diff().dropna()
    
    return(df)

def perform_checks(log_rets, dist_list):

    # Nazwy rozkładow -> obiekty scipy.stats
    distributions_to_check = [getattr(st, distname) for distname in dist_list]
    
    #inicjalizacja pustych tablic
    ks_pvalues = pd.DataFrame() 
    sse=pd.DataFrame()

    best_fits = [] # lista najlepszych (ze względu na SSE) rozkładów dla akcji
    param_list = [] # lista parametrów dopasowanych rozkładów

    for stock in log_rets.columns:
        stock_logrets = log_rets[stock]
        best_sse = np.inf # inicjowanie śmiesznie wysokiego SSE
        best_fit = st.uniform() # whatever, z takim bzdurnym SSE i tak za chwilę będzie nadpisany
        for dist in distributions_to_check:
            params = dist.fit(stock_logrets) # dopasowywanie parametrów rozkładu z rodziny
            loc = params[-2]
            scale = params[-1]
            additional_params = params[:-2] # czasem są, czasem nie (zależnie od rozkładu)
            param_list.append((stock, dist.name, *params)) # zapamiętujemy, na potem


            cdf = lambda x : dist.cdf(x, loc = params[-2], scale = params[-1], *additional_params) # stworzenie funkcji CDF
            ks_pvalues.loc[stock,dist.name] = st.kstest(rvs = stock_logrets,cdf = cdf)[1] # KS-test zgodności próby z rozkładem

            ecdf = ECDF(stock_logrets)
            sse_ = sum((ecdf(stock_logrets)-cdf(stock_logrets))**2) # SSE calc
            sse.loc[stock,dist.name]= sse_ #zapisz SSE

            # Czy ten fit ma lepsze SSE? Jak tak to podmień 
            if sse_<best_sse:
                best_sse = sse_
                best_fit = (dist.name,dist(loc = params[-2], scale = params[-1], *additional_params))
        best_fits.append(best_fit)

    best_fits = dict(zip(log_rets.columns, best_fits)) # dictionary, {akcja -> best_fit}
    
    return(best_fits, ks_pvalues, sse)

def plot_stock_logrets(df, ticker_list):
    # Wykres sum kumulatywnych logzwrotów
    
    df_cumsum = df.cumsum()
    df_cumsum.reset_index(drop=False, inplace=True)
       
    stock_plot = px.line(df_cumsum, x='Date', y=ticker_list, labels=dict(x="Data", y="Log-zwrot"),template=plotly_template)
    stock_plot.update_layout(legend_title_text='Akcja')
    stock_plot.update_yaxes(title_text = "Log-zwrot")
    return stock_plot

def plot_best_dist(ticker, best_fits, log_rets):
    # Wykres best fit (histogram+pdf)
    
    best_dist = best_fits[ticker] # obiekt scipy.stats, best fit dla akcji
    stock_logrets = log_rets[ticker] # logzwroty
    xaxis = np.linspace(min(stock_logrets), max(stock_logrets),100) #jakaś sensowna oś

    best_pdf = best_dist[1].pdf(xaxis) #policz teoretyczny pdf

    #histogram logzwrotów, unormowany
    fig = px.histogram(log_rets, x=ticker, histnorm='probability density',template=plotly_template,
                       title="Najlepsze dopasowanie dla {}: <br> rozkłady {}".format(ticker,best_dist[0]))
    # dorzuć pdf
    fig.add_trace(go.Scatter(x=xaxis,y=best_pdf,mode="lines",line=go.scatter.Line(color="green"),showlegend=False))
    fig.update_yaxes(automargin=True)
    return fig
    
    

def visualize_fitted_families(best_fits, ks_results, tickers, all_possible_dists):
    # wykres słupkowy wyników SSE i KS-testu
    
    # dwa wykresy obok siebie
    specs=[[{"type": "bar"},{"type": "pie"}]]
    
    fig = make_subplots(rows=1, cols=2,
                        specs=specs,
                        subplot_titles=("Odrzucenia KS","Najniższe SSE"))
    
    # lista pt. 'najnizsze SSE' dla każdej akcji
    best_family = [best_fits[ticker][0] for ticker in tickers]
    
    # Ile razy się pojawiała każda rodzina?
    counts = [best_family.count(dist_name) for dist_name in all_possible_dists]
    
    # Zbieramy to do tablicy
    best_sse = pd.DataFrame({'Family':all_possible_dists,
                             'Count':counts})
    # Wykres słupkowy
    #fig.add_trace(go.Bar(x=list(best_sse['Family']), 
    #                     y=list(best_sse['Count'])),
    #              row=1, col=1)
    
    fig.add_trace(go.Pie(automargin=True,
                         values=list(best_sse['Count']),
                         labels=list(best_sse['Family']),
                        showlegend= True,),
                  row=1, col=2)

    
    # Ile razy pwartość KS < 0.05?
    ks_rejections = (ks_results<0.05).sum()
    fig.add_trace(go.Bar(x=list(ks_rejections.index), 
                         y=list(ks_rejections),
                        showlegend=False),
                  row=1, col=1)
    
    
    fig.update_layout(template=plotly_template)
    fig.update_yaxes(automargin=True)
    #fig.layout.annotations[1].update(x=0.6)

    return fig
app.run_server(mode='external')